In [85]:
import datetime
import time
import csv
            
class OV2640Program:
    'Defines an ordered set of register loads for the OV2640 image sensor'
    registerSet0 = [None] * 256
    registerSet1 = [None] * 256
    registerFormat = '0x{:02x}'

    @staticmethod
    def loadRegisters():
        OV2640Register.reserveCounters[0] = 0
        OV2640Register.reserveCounters[1] = 0
        firstRow = True
        with open("Register Map1.csv") as f:
            reader = csv.reader(f)
            for row in reader:
                if firstRow :
                    firstRow = False
                    continue
                address = row[0].strip()
                bank = int(row[1].strip(),10)
                name = row[2].strip()
                subscript = row[3].strip()
                default = row[4].strip()
                direction = row[5].strip()
                mitTag = ''
                bitfields = []
                optCount = len(row)
                if optCount > 6:
                    mitTag = row[6]
                    fieldIndex = 7
                    while fieldIndex < optCount and len(row[fieldIndex]) > 0:
                        bf = OV2640BitField(row[fieldIndex],row[fieldIndex + 1],row[fieldIndex + 2]);
                        bf.compile()
                        bitfields.append(bf)
                        fieldIndex += 3
                dashIndex = address.find("-")
                if dashIndex != -1 :
                    min = int(address[:dashIndex].strip(),16)
                    max = int(address[1+dashIndex:].strip(),16)
                else:
                    min = int(address,16)
                    max = min

                bracketIndex = name.find("[")
                datatype = "WORD"
                if bracketIndex != -1 :
                    bracket = name[bracketIndex:].strip()
                    datatype = types.get(bracket, bracket)
                    name = name[:bracketIndex]
                    
                for regAddr in range(min,max+1):
                    register = OV2640Register(bank,name,subscript,datatype,default,regAddr,mitTag,bitfields)
                    if bank == 0:
                        OV2640Program.registerSet0[regAddr] = register
                    else:
                        OV2640Program.registerSet1[regAddr] = register
    @staticmethod
    def printRegisters(progFile):
        for i in range(0,256):
            if OV2640Program.registerSet0[i] != None:
                print(OV2640Program.registerSet0[i].registerDeclaration(),file=progFile)
        for i in range(0,256):
            if OV2640Program.registerSet1[i] != None:
                print(OV2640Program.registerSet1[i].registerDeclaration(),file=progFile)
    @staticmethod
    def startBracket(fieldFormat):
        if fieldFormat == "C++":
            return "{";
        if fieldFormat == "JSON":
            return "[";
        return "{"
    @staticmethod
    def endBracket(fieldFormat):
        if fieldFormat == "C++":
            return "}";
        if fieldFormat == "JSON":
            return "]";
        return "}"  

    def __init__(self):
        self.program = []
        self.activeBank = None

    def describe(self,programName,fieldFormat,progFile) :
        if fieldFormat == "C++" :
            print("/********************************************",file=progFile)
            print("START Program,{:s}\t\tGenerated,{:s}\t\tLevel:{:s}".format(programName,time.strftime("%c"),fieldFormat),
                  file=progFile)
            print("********************************************/",file=progFile)
            print(file=progFile)
            print("const struct sensor_reg {:s}[] =".format(programName),file=progFile)
            print("{",file=progFile)
        elif fieldFormat == "JSON":
            print("{\n\t\"pgm\" : [",file=progFile)
        fileName = programName + '.csv'
        with open(fileName) as f:
            reader = csv.reader(f)
            self.activeBank = 1
            lineCtr = 1
            for row in reader:
                if len(row) == 2 :
                    regAddr = int(row[0].strip(),16)
                    regValue = int(row[1].strip(),16)
                    assert regAddr != 255 or regValue != 255
                    self.program.append([regAddr,regValue])
                    progtext = self.printLine(regAddr,regValue,fieldFormat,lineCtr)
                        
                    print(progtext,file=progFile)
                    lineCtr += 1
            # too much trouble to generalize yet
            if fieldFormat == "C++":
                print("\t{0xFF,0xFF}",file=progFile)
                print("};",file=progFile)
            elif fieldFormat == "JSON" :
                print("\t[\"0xFF\",\"0xFF\"]",file=progFile)
                print("\t]",file=progFile)
                print("}",file=progFile)
            print(file=progFile)

    def printLine(self,regAddr,regValue,fieldFormat,lineCtr) :
        if regAddr == 255:
            self.activeBank = regValue
            register = OV2640Program.registerSet0[255]
        elif self.activeBank == 0:
            register = OV2640Program.registerSet0[regAddr]
        elif self.activeBank == 1:
            register = OV2640Program.registerSet1[regAddr]
        else:
            opCode = OV2640Program.registerFormat.format(regAddr)
        #lets see about getting a high level definition for this register setting
        if self.activeBank != None :
            if register.hasDescription() :
                printReg = register.mitTag
            else: 
                printReg = register.registerName();
            if(printReg == None or len(printReg) == 0) :
                printReg = OV2640Program.registerFormat.format(regAddr)
            decodedFields,remainingbits = register.decodeArgument(regValue)
            if len(decodedFields) > 0 :
                printVal = register.printArgument(decodedFields)
                if remainingbits > 0 :
                    unknownBits = ' ??{:02x}?? '.format(remainingbits)
                    printVal = printVal + unknownBits
            else:
                printVal = OV2640Program.registerFormat.format(regValue)
        else:
            printReg = OV2640Program.registerFormat.format(regAddr)
            printVal = OV2640Program.registerFormat.format(regValue)
        codeVal = OV2640Program.registerFormat.format(regValue)
        # syntax fixup
        if fieldFormat == "C++":
            codeReg = register.registerName();
            codeVal = OV2640Program.registerFormat.format(regValue)
            codetext = "\t{:s}{:s},{:s}{:s},".format(OV2640Program.startBracket(fieldFormat),
                                                     codeReg,codeVal,OV2640Program.endBracket(fieldFormat))
            commenttext = "{:d}\t[{:02x} := {:02x}]\t\t{:s}\t{:s}".format(lineCtr,regAddr,regValue,printReg,printVal)
            return "{:50s}/*{:s}*/".format(codetext,commenttext)
        elif fieldFormat == "JSON":
            codeReg = "\"" + OV2640Program.registerFormat.format(regAddr) + "\""
            codeVal = "\"" + OV2640Program.registerFormat.format(regValue) + "\""
            commentText = "\"" + "{:03d}   {:s} = {:s}".format(lineCtr,printReg,printVal) + "\""
            codetext = "\t{:s}{:s},{:s},{:s}{:s},".format(OV2640Program.startBracket(fieldFormat),
                                                          codeReg,codeVal,commentText,OV2640Program.endBracket(fieldFormat))
            return codetext
        # \t {regname,hexvalue},\t/*Line#\tdesc*/
        
 
        return 
 

In [86]:
class OV2640BitField :
    def __init__(self,name,value,desc):
        self.name = name.strip()
        self.value = value.strip()
        self.description = desc.strip()
        self.isVariable = None
        self.isParsed = False
    def hasDescription(self) :
        return self.description != None and len(self.description) > 0
    # return description or name
    def getDescription(self) :
        if self.hasDescription() :
            return self.description
        return self.name
    # return the field in a non-compilable descriptive form
    # format H)igh level, L)ow level, M)achine level
    def printField(self,decodedArg) :
        # highlevel\t\tlow level
        highLevel = ""
        if self.isSimpleFlag() and self.hasDescription():
            highLevel = '{:s}'.format(self.getDescription())
        else:
            highLevel = '{:s}({:02x})'.format(self.getDescription(),decodedArg)

        if self.isSimpleFlag() :
            desc = '{:s}'.format(self.name)
        else:
            desc = '{:s}({:d})'.format(self.name,decodedArg)

            
        return highLevel  +"/" + desc

    #true for constant flags
    def isSimpleFlag(self) :
        return self.isVariable == False
    # returns the flag value
    def simpleFlagValue(self,progArg) :
        if ((progArg & self.value == self.value and self.value > 0) or
            (progArg == 0 and self.value == 0)):
            return self.value
        return None
    # true if the bit field is set
    def isBitField(self) :
        return self.isVariable == True
    # get the bit field value AND mask it out
    def extractBitFieldValue(self,progArg) :
        shiftmask = self.value[1] << self.value[2]
        bits = progArg & shiftmask
        downshift = self.getRegisterShift()
        bits = bits >> downshift
        bits = bits & self.value[1]
        bits << self.value[0]
        #mask the field out of the arg
        progArg = progArg & (~shiftmask)
        return [bits,progArg]
    def getRegisterShift(self):
        if self.isVariable != True:
            return None
        return self.value[2]
    def compile(self):
        if self.isParsed == False and self.value.startswith('0x'):
            self.value = int(self.value,16)
            self.isVariable = False
            self.isParsed = True
        elif self.isParsed == False and self.value.startswith('[') :
            ss = self.value[1:-1]
            ss = ss.split(',')
            ss[0] = int(ss[0],10)
            ss[1] = int(ss[1],16)
            ss[2] = int(ss[2],10)
            self.value = ss
            self.isVariable = True


In [87]:
class OV2640Register:
    'Defines a register on the OV2640 Sensor Array'
    reserveCounters = [0,0]
    def __init__(self,bank,name,subscript,datatype,defaultValue,address,mitTag,bitFields) :
        self.bank = bank
        if name == 'RSVD':
            c = address
            name = '{:s}_{:02x}'.format(name,c)
            c += 1
            OV2640Register.reserveCounters[self.bank] = c
        
        self.name = name
        self.subscript = subscript
        self.datatype = datatype
        self.defaultValue = defaultValue
        self.address = address
        self.mitTag = mitTag
        self.bitFields = bitFields
    def hasDescription(self) :
        return self.mitTag != None and len(self.mitTag) > 0
    def registerName(self) :
        return 'RB{:d}_{:s}'.format(self.bank,self.name)
    
    # Used for C++ register declarations
    def registerDeclaration(self) :
        declformat = 'const uint8_t {:s} = ' + OV2640Program.registerFormat + ';'
        return declformat.format(self.registerName(),self.address)
    def cantDecodeArgument(self,progArg) :
        return OV2640Program.registerFormat.format(progArg)
    # this will example a register value and attempt to decode it - it
    # will return any bitfields and flags it identifies and the remaining
    # bits that were not processed
    def decodeArgument(self,progArg):
        selectedFields = []      
        # if no bitfield definitions then it can't be decoded
        if(len(self.bitFields) == 0):
            return [selectedFields,progArg]
        
        # iterate over the bit fields
        for aBitField in self.bitFields :
            if aBitField.isSimpleFlag() :
                flagValue = aBitField.simpleFlagValue(progArg)
                if flagValue != None :
                    selectedFields.append([aBitField,flagValue])
                # mask the field out of the arg
                progArg = progArg & (~aBitField.value)
            elif aBitField.isBitField() :
                fieldVal,newProgArg = aBitField.extractBitFieldValue(progArg)
                selectedFields.append([aBitField,fieldVal])
                #mask the field out of the arg
                progArg = newProgArg
        return [selectedFields,progArg]
                                       
    
    def printArgument(self,decodeVector) :
        summary = ''
        for decodeItem in decodeVector :
            bitField,fieldValue = decodeItem
            if len(summary) > 0:
                summary += " | "
            dtext = bitField.printField(fieldValue)
            desc = ' {:s} '.format(dtext)
            summary = summary + desc
        return summary


In [88]:
OV2640Program.loadRegisters()

progFile = open("registerDefinitions.h", "w")
OV2640Program.printRegisters(progFile)
progFile.close()
programs = ['OV2640_QVGA','OV2640_JPEG_INIT','OV2640_YUV422','OV2640_JPEG','OV2640_160x120_JPEG','OV2640_176x144_JPEG',
            'OV2640_320x240_JPEG','OV2640_352x288_JPEG','OV2640_640x480_JPEG','OV2640_800x600_JPEG',
            'OV2640_1024x768_JPEG','OV2640_1280x1024_JPEG','OV2640_1600x1200_JPEG']
for aProgram in programs :
    pgm = OV2640Program();
    progFile = open(aProgram + ".regpgm.h", "w")
    pgm.describe(aProgram,"C++",progFile)
    progFile.close()
    progFile = open(aProgram + ".regpgm.json", "w")
    pgm.describe(aProgram,"JSON",progFile)
    progFile.close()

In [96]:

types = {"[7:0]": "BYTE","[3:0]":"NIBBLE","[1:0]":"BIT"}


OV2640Program.loadRegisters()
#OV2640Program.printRegisters()
pgm = OV2640Program();
#pgm.compile('OV2640_QVGA')
pgm.compile('OV2640_JPEG_INIT')
pgm.compile('OV2640_YUV422')
pgm.compile('OV2640_JPEG')
pgm.compile('OV2640_160x120_JPEG')
pgm.compile('OV2640_176x144_JPEG')
pgm.compile('OV2640_320x240_JPEG')
pgm.compile('OV2640_352x288_JPEG')
pgm.compile('OV2640_640x480_JPEG')
pgm.compile('OV2640_800x600_JPEG')
pgm.compile('OV2640_1024x768_JPEG')
pgm.compile('OV2640_1280x1024_JPEG')
pgm.compile('OV2640_1600x1200_JPEG')

Compiling OV2640_JPEG_INIT
const uint8_t[] OV2640_JPEG_INIT {
	RB0_RA_DLMT,DSP,
	RB0_RSVD43,0xff,
	RB0_RSVD45,0xdf,
	RB0_RA_DLMT,SENS,
	RB1_RSVD27,0x32,
	RB1_CLKRC,0x04,
	RB1_COM2,0x02,
	RB1_REG04,0x28,
	RB1_COM8,DEF | BNDF_EN | AGC_EN | AEC_EN,
	RB1_COM9,AGC_GAIN_8x,
	RB1_RSVD18,0x0c,
	RB1_RSVD19,0x78,
	RB1_RSVD25,0x33,
	RB1_RSVD26,0xfb,
	RB1_RSVD29,0x00,
	RB1_RSVD34,0x11,
	RB1_RSVD07,0x10,
	RB1_RSVD24,0x92,
	RB1_RSVD20,0xda,
	RB1_RSVD13,0x1a,
	RB1_RSVD22,0xc3,
	RB1_RSVD14,0x00,
	RB1_ARCOM2,0xc0,
	RB1_RSVD21,0x1a,
	RB1_RSVD03,0x88,
	RB1_RSVD04,0xc0,
	RB1_COM4,0x87,
	RB1_RSVD05,0x41,
	RB1_RSVD37,0x00,
	RB1_COM19,0x00,
	RB1_RSVD49,0x00,
	RB1_RSVD33,0x03,
	RB1_RSVD36,0x81,
	RB1_RSVD12,0x99,
	RB1_AEW,0x40,
	RB1_AEB,0x38,
	RB1_VV,0x82,
	RB1_RSVD50,0x00,
	RB1_RSVD51,0x00,
	RB1_HISTO_LOW,0x70,
	RB1_HISTO_HIGH,0x80,
	RB1_RSVD76,0x05,
	RB1_RSVD11,0x80,
	RB1_RSVD16,0x30,
	RB1_RSVD60,0x00,
	RB1_RSVD61,0x80,
	RB1_RSVD62,0x00,
	RB1_RSVD64,0x02,
	RB1_RSVD65,0x94,
	RB1_RSVD67,0xc1,
	RB1_COM7,RES_SVG

In [37]:
OV2640Program.loadRegisters()

bitfields = 0
DSP_BYPAS
USE_DSP
bitfields = 2
bitfields = 0
bitfields = 0
bitfields = 0
LP_DP
ROUND
V_DIV_SET(x)
H_DIV_SET(x)
bitfields = 4
SET(x)
bitfields = 1
SET(x)
bitfields = 1
bitfields = 0
SET(x)
bitfields = 1
VSIZE_SET(x)
HSIZE_SET(x)
YOFF_SET(x)
XOFF_SET(x)
bitfields = 4
bitfields = 0
HSIZE_SET(x)
bitfields = 1
OUTW_SET(x)
bitfields = 1
OUTH_SET(x)
bitfields = 1
ZSPEED_SET(x)
OUTH_SET(x)
OUTW_SET(x)VAL_SET(x, 0x3, (8+2), 0)
bitfields = 3
bitfields = 0
bitfields = 0
bitfields = 0
bitfields = 0
DCW_EN
SDE_EN
UV_ADJ_EN
UV_AVG_EN
CMX_EN
bitfields = 5
BPC_EN
WPC_EN
bitfields = 2
bitfields = 0
HSIZE8_11_SET(x)
HSIZE8_SET(x)
VSIZE8_SET(x)
bitfields = 3
bitfields = 0
SET(x)
bitfields = 1
SET(x)
bitfields = 1
AEC_EN
AEC_SEL
STAT_SEL
VFIRST
YUV422
YUV_EN
RGB_EN
RAW_EN 
bitfields = 8
CIP
DMY
RAW_GMA
DG
AWB
AWB_GAIN
LENC
PRE
bitfields = 8
bitfields = 0
SP_AUTO_MODE
SP_DVP_MASK
bitfields = 2
bitfields = 0
Y8_DVP_EN
JPEG_EN
YUV422
RAW10
RGB565
HREF_VSYNC
LBYTE_FIRST
bitfields = 7
bitfields 

IndexError: list index out of range